# Hakai Nutrient QA-QC 
---
# Start Notebook
## Install packages 
Google colab servers has already a few commonly used packages installed. To install those missing, which are specific toh Hakai let's run the following commands:

In [3]:
%pip install git+https://github.com/HakaiInstitute/hakai-oceanography-qc-tools.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/HakaiInstitute/hakai-oceanography-qc-tools.git to /tmp/pip-req-build-23d9zwpu
  Running command git clone --filter=blob:none --quiet https://github.com/HakaiInstitute/hakai-oceanography-qc-tools.git /tmp/pip-req-build-23d9zwpu
  Resolved https://github.com/HakaiInstitute/hakai-oceanography-qc-tools.git to commit af281d2dd7c871dc15e3e1c528fe7795ba938961
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 KB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 KB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 555.3/555.3 KB 42.5 MB/s eta 0:00:00
  Created wheel for hakai-qc: filename=hakai_qc-1.0-py3-none-any.whl size=2728 sha256=1c03be5bfabae54936e659431cd44507c9e99a0da5c282a1d50c037743ea8bf6
  St

## Import the python packages needed

In [2]:
# Let's load pandas for working with the data in table
import pandas as pd 
import numpy as np

# Let's load seaborn to plot the data
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# This to install the hakai api tool and be able to download some data from hakai's database
from hakai_api import Client

# Install ioos_qc which is used to qc data
from ioos_qc import qartod

# from ipywidgets import interact, interactive, fixed, interact_manual
# from ipywidgets import widgets, HBox, VBox
# import time

from hakai_qc.flags import flag_color_map, flag_qartod_to_hakai
from  hakai_qc.ioos_qc import qc_dataframe

# Define some project standards and get Hakai API credentials 

In [3]:
# And the interesting variables to use from the Hakai database
nutrient_variables = ['no2_no3_um','sio2','po4']

pd.options.display.max_columns = 100
client = Client()

# Download data from the Hakai database
For more information regarding the Hakai API, go [here](https://github.com/HakaiInstitute/hakai-api).

You can find a list of all the data type endpoints [here](http://hakaiinstitute.github.io/hakai-api/#endpoints).

In [4]:
# Let's retrieve the endpoint to retrieve nutrients data:
endpointUrl = '/eims/views/output/nutrients'
site_id = 'QU39'
start_time = '2012-01-01'
end_time = '2022-06-11'
# We'll retrieve data only associated with QU39 between January 1st 2019 to January 1st 2020
filterUrl = 'site_id={0}&collected>{1}&collected<{2}&limit=-1'.format(
    site_id, start_time, end_time
)

# Make a data request for sampling stations
url = f'{client.api_root}/{endpointUrl}?{filterUrl}'
response = client.get(url)
df = pd.DataFrame(response.json())
df_original = df.copy()
original_columns = df.columns
print(f'{len(df)} records downloaded')
df.head()

3909 records downloaded


,action,event_pk,rn,is_replicate,date,work_area,organization,project,survey,sampling_bout,site_id,project_specific_id,hakai_id,source,lat,long,gather_lat,gather_long,collection_method,line_out_depth,pressure_transducer_depth,filtered,filter_type,volume,installed,collected,preserved,analyzed,lab_technician,nh4_,no2_no3_um,no2_no3_ugl,no2_no3_units,tp,tdp,tn,tdn,srp,po4,sio2,po4pfilt,no3nfilt,po4punfl,no3nunfl,nh4nunfl,nh4__flag,no2_no3_flag,tp_flag,tdp_flag,tn_flag,tdn_flag,srp_flag,po4_flag,sio2_flag,po4pfilt_flag,no3nfilt_flag,po4punfl_flag,no3nunfl_flag,nh4nunfl_flag,analyzing_lab,row_flag,metadata_qc_flag,quality_level,comments,quality_log
0,,416,1,None,2015-03-18,QUADRA,HAKAI,"OCEANOGRAPHY,OCEANOGRAPHY,OCEANOGRAPHY","QOMA1,QOMA2,QOMA",1,QU39,QNUT382,QNUT382,M,50.0307,-125.0992,NaN,NaN,None,0,NaN,None,0.45nm,13,None,2015-03-18T16:02:55.000Z,2015-03-18T16:02:14.000Z,2015-05-21T07:00:00.000Z,"Kate,Katie,Rebecca",None,5.918382,None,uM,None,None,None,None,None,0.637,12.324552,None,None,None,None,None,None,AV,AV,AV,AV,AV,AV,AV,AV,None,None,None,None,None,UBC,Results,None,Technicianmr,,1: Every sample taken today that says station ...
1,,416,1,None,2015-03-18,QUADRA,HAKAI,"OCEANOGRAPHY,OCEANOGRAPHY,OCEANOGRAPHY","QOMA1,QOMA2,QOMA",1,QU39,QNUT383,QNUT383,M,50.0307,-125.0992,NaN,NaN,None,5,NaN,None,0.45nm,13,None,2015-03-18T16:02:55.000Z,2015-03-18T16:02:14.000Z,2015-05-21T07:00:00.000Z,"Kate,Katie,Rebecca",None,6.011794,None,uM,None,None,None,None,None,0.638,12.396646,None,None,None,None,None,None,AV,AV,AV,AV,AV,AV,AV,AV,None,None,None,None,None,UBC,Results,None,Technicianmr,,1: Every sample taken today that says station ...
2,,416,1,None,2015-03-18,QUADRA,HAKAI,"OCEANOGRAPHY,OCEANOGRAPHY,OCEANOGRAPHY","QOMA1,QOMA2,QOMA",1,QU39,QNUT384,QNUT384,M,50.0307,-125.0992,NaN,NaN,None,10,NaN,None,0.45nm,13,None,2015-03-18T16:02:55.000Z,2015-03-18T16:02:14.000Z,2015-05-21T07:00:00.000Z,"Kate,Katie,Rebecca",None,6.634283,None,uM,None,None,None,None,None,0.700,12.694025,None,None,None,None,None,None,AV,AV,AV,AV,AV,AV,AV,AV,None,None,None,None,None,UBC,Results,None,Technicianmr,,1: Every sample taken today that says station ...
3,,416,1,None,2015-03-18,QUADRA,HAKAI,"OCEANOGRAPHY,OCEANOGRAPHY,OCEANOGRAPHY","QOMA1,QOMA2,QOMA",1,QU39,QNUT385,QNUT385,M,50.0307,-125.0992,NaN,NaN,None,30,NaN,None,0.45nm,13,None,2015-03-18T16:02:55.000Z,2015-03-18T16:02:14.000Z,2015-05-21T07:00:00.000Z,"Kate,Katie,Rebecca",None,24.151575,None,uM,None,None,None,None,None,2.085,42.222115,None,None,None,None,None,None,AV,AV,AV,AV,AV,AV,AV,AV,None,None,None,None,None,UBC,Results,None,Technicianmr,,1: Every sample taken today that says station ...
4,,416,1,None,2015-03-18,QUADRA,HAKAI,"OCEANOGRAPHY,OCEANOGRAPHY,OCEANOGRAPHY","QOMA1,QOMA2,QOMA",1,QU39,QNUT386,QNUT386,M,50.0307,-125.0992,NaN,NaN,None,50,NaN,None,0.45nm,13,None,2015-03-18T16:02:55.000Z,2015-03-18T16:02:14.000Z,2015-05-21T07:00:00.000Z,"Kate,Katie,Rebecca",None,26.960609,None,uM,None,None,None,None,None,2.251,49.347595,None,None,None,None,None,None,AV,AV,AV,AV,AV,AV,AV,AV,None,None,None,None,None,UBC,Results,None,Technicianmr,,1: Every sample taken today that says station ...


In [5]:
# Let's convert the collected time to a datetime object variable called  time 
#  and extract the from those datetime objects the year and month 
df['time'] = pd.to_datetime(df['collected'],utc=True).dt.tz_localize(None)
df['year'] = df['time'].dt.year
df['month'] = df['time'].dt.month
df['dayoftheyear'] = pd.to_timedelta(df['time'].dt.dayofyear, unit='d')

# Define a depth variable which is: 
#   - pressure_transducer_depth (if available)
#   - OR line_out_depth
df['depth'] = df['pressure_transducer_depth'].fillna(df['line_out_depth'])

# Review Replicates
## Pool Standard Deviation


In [66]:
 # Let's create a pooled standard deviation function
def pooled_standard_deviation(df_to_review,count_col='count',std_col='std'):
    # Keep only records that have replicates
    df_replicates = df_to_review[df_to_review[count_col]>1]
    upper =  df_replicates[count_col].sub(1).mul(df_replicates[std_col].pow(2)).sum()
    lower = df_replicates[count_col].sub(1).sum()
    return np.sqrt(upper/lower)

# Get pool standard deviation values from replicates
df_grouped = df.groupby(['site_id','line_out_depth','collected'])[['no2_no3_um','sio2','po4']].agg(['mean','std','count'])

# printresults
for var in nutrient_variables:
    pool_std = pooled_standard_deviation(df_grouped[var])
    print('{0} pool.std.: {1}'.format(var,pool_std))

no2_no3_um pool.std.: 1.2323662019810557
sio2 pool.std.: 2.815441096196155
po4 pool.std.: 0.11119541710739704


In [67]:
def pool_std(stds,counts):
    upper =np.sum(np.subtract(counts,1)*np.power(stds,2))
    lower = np.subtract(np.sum(counts),len(counts))
    return np.sqrt((upper/lower))

def pool_std(stds,counts):
    upper = (counts.sub(1).mul(stds.pow(2))).sum()
    lower = (counts.sum()-len(counts)).sum()
    return np.sqrt(upper/lower)

pool_std(pd.Series([4.222321,5.823036,5.249778]),pd.Series([20,75,35]))

5.459190774324271

In [15]:
df_grouped

no2_no3_um                  \
                                                      mean       std count   
site_id line_out_depth collected                                             
QU39    0              2015-03-18T16:02:55.000Z   5.918382       NaN     1   
                       2015-03-24T16:04:16.000Z  12.621053  0.120735     2   
                       2015-03-31T15:57:34.000Z  15.233539       NaN     1   
                       2015-04-06T15:44:21.000Z  13.544474       NaN     1   
                       2015-04-14T15:33:48.000Z   1.051300       NaN     1   
...                                                    ...       ...   ...   
        265            2016-03-15T16:24:07.000Z  30.436727       NaN     1   
                       2016-03-22T16:37:08.000Z  30.497000       NaN     1   
                       2016-03-29T15:53:53.000Z  31.301056       NaN     1   
                       2016-04-14T16:09:43.000Z  30.596418       NaN     1   
                       2016-04-28T15:21:19.000Z  31.177459       NaN     1   

                                                      sio2                 \
                                                      mean      std count   
site_id line_out_depth collected                                            
QU39    0              2015-03-18T16:02:55.000Z  12.324552      NaN     1   
                       2015-03-24T16:04:16.000Z  19.961093  0.21161     2   
                       2015-03-31T15:57:34.000Z  25.697697      NaN     1   
                       2015-04-06T15:44:21.000Z  23.934000      NaN     1   
                       2015-04-14T15:33:48.000Z   4.693610      NaN     1   
...                                                    ...      ...   ...   
        265            2016-03-15T16:24:07.000Z  65.235003      NaN     1   
                       2016-03-22T16:37:08.000Z  63.091582      NaN     1   
                       2016-03-29T15:53:53.000Z  64.982489      NaN     1   
                       2016-04-14T16:09:43.000Z  63.311283      NaN     1   
                       2016-04-28T15:21:19.000Z  61.352219      NaN     1   

                                                      po4                  
                                                     mean       std count  
site_id line_out_depth collected                                           
QU39    0              2015-03-18T16:02:55.000Z  0.637000       NaN     1  
                       2015-03-24T16:04:16.000Z  1.401500  0.012021     2  
                       2015-03-31T15:57:34.000Z  1.531128       NaN     1  
                       2015-04-06T15:44:21.000Z  1.314000       NaN     1  
                       2015-04-14T15:33:48.000Z  0.161158       NaN     1  
...                                                   ...       ...   ...  
        265            2016-03-15T16:24:07.000Z  2.776677       NaN     1  
                       2016-03-22T16:37:08.000Z  2.797892       NaN     1  
                       2016-03-29T15:53:53.000Z  2.858922       NaN     1  
                       2016-04-14T16:09:43.000Z  2.884316       NaN     1  
                       2016-04-28T15:21:19.000Z  2.860584       NaN     1  

[3331 rows x 9 columns]

In [69]:
df_stats = df_grouped.swaplevel(-1,0,axis='columns')
df_stats.loc[(df_stats['count']>1).all(axis='columns')].groupby(level=1).apply(lambda x: pool_std(x.std,x.count))

AttributeError: 'function' object has no attribute 'sub'

## Replicates Standard Deviation Distribution
Present the distribution of the standard deviations the replicate samples for each line_out_depth.

In [9]:
replicates = df_grouped.stack(level=0).dropna(subset=['std']).sort_index(level=1)
replicates.index.names = ['site_id', 'line_out_depth', 'collected', 'nutrients']

fig = px.histogram(replicates.reset_index(),
              x='std',color='line_out_depth', hover_name='collected',
              facet_row='nutrients')
fig.update_layout(width=1400, height=1500)
fig.update_xaxes(matches=None)
fig.for_each_xaxis(lambda xaxis: xaxis.update(showticklabels=True)) 
fig.update_yaxes(matches=None)
fig.for_each_yaxis(lambda yaxis: yaxis.update(showticklabels=True))
fig.show()

# Time series QARTOD tests
Review each depth time series for a station and run timeseries specific test on them. The configuarion dictionary below, list of each variables and depth ranges the tests that will be applied.


## Set QARTOD Tests Configuration

In [10]:
nutrients_qc_configs= {
    "-5 < depth < 50": """
        contexts:
                -   window:
                        starting: 2010-01-01T00:00:00Z
                        ending: null
                    streams:
                        no2_no3_um:
                            qartod:
                                gross_range_test:
                                    suspect_span: [0, 36]
                                    fail_span: [0, 40]
                        po4:
                            qartod:
                                gross_range_test:
                                    suspect_span: [0, 3]
                                    fail_span: [0, 4]
                        sio2:
                            qartod:
                                gross_range_test:
                                    suspect_span: [0,80]
                                    fail_span: [0,100]
        """
    ,
    "50<=depth": """
        contexts:
             -  window:
                    starting: 2010-01-01T00:00:00Z
                    ending: null
                streams:
                    no2_no3_um:
                        qartod:
                            gross_range_test:
                                suspect_span: [0, 36]
                                fail_span: [0, 40]
                            spike_test:
                                suspect_threshold: 2
                                fail_threshold: 3
                                method: 'differential'
                    po4:
                        qartod:
                            gross_range_test:
                                suspect_span: [0, 3]
                                fail_span: [0, 4]
                            spike_test:
                                suspect_threshold: 0.2
                                fail_threshold: 0.4
                                method: 'differential'
                    sio2:
                        qartod:
                            gross_range_test:
                                suspect_span: [0,80]
                                fail_span: [0,100]
                            spike_test:
                                suspect_threshold: 8
                                fail_threshold: 12
                                method: 'differential'
        """
}

## Run QARTOD Tests on Depth TimeSeries


In [11]:
# Run QARTOD tests
overwrite_existing_flags = False
df = qc_dataframe(df.drop(columns=[col for col in df.columns if 'qartod' in col]),configs=nutrients_qc_configs,groupby=['site_id','line_out_depth'])
df = df.set_index(['index']).sort_index()

# aggregate flags
for var in ['no2_no3_um','po4','sio2']:
    agg_flag = f"{var}_qartod_aggregate"
    df.loc[:,agg_flag] = qartod.qartod_compare(df.filter(like=var+'_qartod_').fillna(9).astype(int).transpose().to_numpy())
    # Map to hakai convention and update empty flags
    var_flag = "no2_no3_flag" if var == 'no2_no3_um' else f"{var}_flag"
    if overwrite_existing_flags:
        df.loc[:,var_flag] = df[agg_flag].replace(flag_qartod_to_hakai)
    else:
        df.loc[:,var_flag] = df[var_flag].fillna(df[agg_flag].replace(flag_qartod_to_hakai))


# Apply lower than detection limit flag as BDL
df.loc[df['no2_no3_um']<0.036 ,'no2_no3_um_flag']='BDL'
df.loc[df['po4']<0.032 ,'po4_flag']='BDL'
df.loc[df['sio2']<0.1 ,'sio2_flag']='BDL'

## Review QARTOD Results

In [12]:
df.columns

Index(['action', 'event_pk', 'rn', 'is_replicate', 'date', 'work_area',
       'organization', 'project', 'survey', 'sampling_bout', 'site_id',
       'project_specific_id', 'hakai_id', 'source', 'lat', 'long',
       'gather_lat', 'gather_long', 'collection_method', 'line_out_depth',
       'pressure_transducer_depth', 'filtered', 'filter_type', 'volume',
       'installed', 'collected', 'preserved', 'analyzed', 'lab_technician',
       'nh4_', 'no2_no3_um', 'no2_no3_ugl', 'no2_no3_units', 'tp', 'tdp', 'tn',
       'tdn', 'srp', 'po4', 'sio2', 'po4pfilt', 'no3nfilt', 'po4punfl',
       'no3nunfl', 'nh4nunfl', 'nh4__flag', 'no2_no3_flag', 'tp_flag',
       'tdp_flag', 'tn_flag', 'tdn_flag', 'srp_flag', 'po4_flag', 'sio2_flag',
       'po4pfilt_flag', 'no3nfilt_flag', 'po4punfl_flag', 'no3nunfl_flag',
       'nh4nunfl_flag', 'analyzing_lab', 'row_flag', 'metadata_qc_flag',
       'quality_level', 'comments', 'quality_log', 'time', 'year', 'month',
       'dayoftheyear', 'depth', 'no2_no

In [16]:
# flag_color_map = {key:value['Color'] for key,value in review.flag_conventions['HAKAI'].items()}

line_out_depths = line_out_depth=df['line_out_depth'].drop_duplicates().sort_values().tolist()
line_out_depth_selector = widgets.SelectMultiple(
    options=line_out_depths,
    value=[line_out_depths[0]],
    description='line_out_depth',
)

def make_qartod_plot_review(var,line_out_depth):
    flag_vars = df.filter(regex='flag|qartod').columns.tolist()
    fig = px.line(df[df['line_out_depth'].isin(line_out_depth)].sort_values(['time',f"{var}_qartod_aggregate"]),
                x='time',y=var,
                color=f"{var}_flag" if var!='no2_no3_um' else "no2_no3_flag", 
                symbol='quality_level',
                hover_data=['hakai_id']+flag_vars,
                color_discrete_map=flag_color_map)
    for trace in fig.data:
        if 'AV' not in trace.name:
            trace.mode = 'markers'

    fig.update_layout(
        height=600,
        width=1200,
    )
    return fig.show()

interact(make_qartod_plot_review, var=['sio2','po4','no2_no3_um'],line_out_depth=line_out_depth_selector)

interactive(children=(Dropdown(description='var', options=('sio2', 'po4', 'no2_no3_um'), value='sio2'), Select…

<function __main__.make_qartod_plot_review(var, line_out_depth)>

# Red field ratio

In [14]:
def get_red_field_plot(var,slope_limit,max_depth):
  labels= {'sio2':f'SiO2 (uM)', 'po4':'PO4 (uM)', 'line_out_depth':'Bottle Target Depth (m)'}
  figs=px.scatter(df.query("line_out_depth<@max_depth"),
                x=var,
                y='no2_no3_um',
                color='line_out_depth', 
                hover_data=['hakai_id','date'], 
                template='simple_white',
                title=labels[var],
                labels=labels, 
                facet_col='year')

  for id, item in enumerate(figs.data):
    figs.add_trace(go.Scatter(x = [0, slope_limit[0]],
                              y =  [0, slope_limit[1]],
                              mode='lines',
                              line_color='red',
                              showlegend=False
                              ),
                  row=1,
                  col=id+1)
  return figs

get_red_field_plot('po4',[2.1875,35],100)

In [15]:
get_red_field_plot('sio2',[32.8125,35],100)

# Review Interannual Variability
Let's compute the average value measured for each depth and the associated standard deviation.

## Compute the seasonal variability
### Development 
Generate for each line_out_depth a seasonal variability model



In [17]:
# Generate a reference depth variable
df['reference_depth'] = df['line_out_depth']
df.loc[df['reference_depth']>230, 'reference_depth'] = 260 # Let's group all data below 230 in the same group
df = df.sort_values('reference_depth')
df['reference_depth'] = df['reference_depth'].astype(str)

In [22]:
# Reseample data to a standard grid for each year starting on Jan 1st. Average values within the same grid
grid_window = '14D'
df_resampled = pd.DataFrame()
resampled = []
groupby = ['site_id','year','reference_depth']
for index,df_group in df.groupby(['site_id','year','reference_depth']):
    df_temp = df_group.resample(grid_window,on='time',origin=pd.to_datetime(f"{index[1]}-01-01 00:00:00")).mean()
    df_temp[groupby] = index
    resampled += [df_temp]

df_resampled = pd.concat(resampled).reset_index()
df_resampled['dayoftheyear'] = df_resampled['time'].dt.dayofyear

# For each similar day of the year compute the interannual variability
df_interannual = df_resampled.drop(columns=['time']).groupby(['site_id','reference_depth','dayoftheyear']).agg(['mean','std']).reset_index()

# Center each window pandas resample give start of the window
df_interannual['dayoftheyear'] = df_interannual['dayoftheyear'] + pd.to_timedelta(grid_window).days/2 


In [23]:
# Generate Upper and Lower limits based on standard deviation
alpha = 2
for var in nutrient_variables:
    df_interannual[(var,'lower_limit')] = df_interannual[var]['mean'] - alpha*df_interannual[var]['std']
    df_interannual[(var,'upper_limit')] = df_interannual[var]['mean'] + alpha*df_interannual[var]['std']

In [24]:
def plot_seasonal(reference_depth,year,variable):
    df_filtered  = df_interannual.loc[df_interannual['reference_depth'].isin(reference_depth)]
    df_data_filtered = df.query(f"reference_depth in {reference_depth} and year in {year}").copy()
    df_data_filtered['dayoftheyear'] = df_data_filtered['dayoftheyear'].dt.days

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_filtered['dayoftheyear'],y=df_filtered[variable]['mean'],name='interannual mean',line_color='black'))
    fig.add_trace(go.Scatter(x=df_filtered['dayoftheyear'],y=df_filtered[variable]['upper_limit'],line_color='lightgrey',mode=None,name=f'interannual mean + {alpha}*STD'))
    fig.add_trace(go.Scatter(x=df_filtered['dayoftheyear'],y=df_filtered[variable]['lower_limit'],line_color='lightgrey',fill='tonexty',mode=None,name=f'interannual mean - {alpha}*STD'))
    fig_data = px.line(df_data_filtered.sort_values(['reference_depth','dayoftheyear']),
                       x='dayoftheyear',y=variable,
                       line_dash='year',
                       color='no2_no3_flag' if variable =='no2_no3_um' else f"{variable}_flag",
                       hover_data=['hakai_id'],
                       color_discrete_map=flag_color_map)
    
    for item in fig_data.data:
        # If AV add a line between markers
        if 'AV' in item['name']:
            item['mode'] = 'lines+markers'
        else:
            item['mode'] = 'markers'

        fig.add_trace(item)
        
    fig.update_yaxes(title=variable)
    fig.update_xaxes(title='Day Of The Year')
    fig.update_layout(width=1500,
                      title='Interannual variability')
    fig.show()

year_selector = widgets.SelectMultiple(
    options=df['year'].sort_values().unique(),
    value=[df['year'].sort_values().unique()[0]],
    description="Select Year(s)",
)
depth_selector = widgets.SelectMultiple(
    options=df['reference_depth'].unique(),
    value=[df['reference_depth'].values[0]],
    description="Select depths",
)
interact(plot_seasonal,
         reference_depth=depth_selector,
         year=year_selector,
         variable=nutrient_variables)

interactive(children=(SelectMultiple(description='Select depths', index=(0,), options=('0', '5', '10', '20', '…

<function __main__.plot_seasonal(reference_depth, year, variable)>

## Make Monthly Box Plots

In [25]:
def make_boxplot_per_depth(var,reference_depth):
    # Show the seasonality of the data, averaged per month for all years
    #  and give the interannual box pot statistic
    fig = px.box(df.query(f"reference_depth == '{reference_depth}'"),
                 x='month',y=var,hover_data=['hakai_id'])
    fig.update_layout(width=1500)
    return fig.show()
    
interact(make_boxplot_per_depth,var=nutrient_variables, reference_depth = df['reference_depth'].unique())

interactive(children=(Dropdown(description='var', options=('no2_no3_um', 'sio2', 'po4'), value='no2_no3_um'), …

<function __main__.make_boxplot_per_depth(var, reference_depth)>

# Create Aggregated Suggested Flag


## Review Profiles

In [26]:
# Plot profile
group_profile_by = 'date'
profile_list = widgets.SelectMultiple(
    options=df[group_profile_by].sort_values().drop_duplicates(),
    value=(df[group_profile_by].iloc[0],),
    description="Select Profiles",
    disabled=False,
)
def make_profile_plot(profiles):
    # Filter only selected profiles
    # df_selected = df[df['date'].isin(profiles)]
    df_temp = df[df['date'].isin(profiles)].sort_values(['date','depth'])
    fig = []
    for var in nutrient_variables:
        fig.append(px.line(df_temp,x=var,y='depth',color=f"{var}_flag" if var!= "no2_no3_um" else "no2_no3_flag",hover_data=['hakai_id'], color_discrete_map=flag_color_map))

    subfig = make_subplots(rows=1, cols=3)
    kk=1
    for item in fig:
        for subitem in item.data:
            if kk>1:
                subitem.showlegend=False
            subfig.add_trace(subitem,row=1,col=kk)
        subfig.update_xaxes(title_text=nutrient_variables[kk-1], row=1, col=kk)
        kk += 1
    subfig.update_yaxes(title_text='Depth (m)', row=1, col=1)
    subfig.update_yaxes(autorange="reversed")
    subfig.update_xaxes(matches=None)
    subfig.update_traces(mode='markers+lines')
    subfig.update_layout(width=1500)
    return subfig

interact(make_profile_plot, profiles=profile_list)

interactive(children=(SelectMultiple(description='Select Profiles', index=(0,), options=('2015-03-18', '2015-0…

<function __main__.make_profile_plot(profiles)>

## Review Timeseries

# Get List of Sample Flagged

In [27]:
# Show me the data flagged
df.loc[df[['no2_no3_flag','sio2_flag','po4_flag']].isin(['SVC','SVD','BDL']).any(axis='columns')][['date','hakai_id','no2_no3_flag','sio2_flag','po4_flag']]

,date,hakai_id,no2_no3_flag,sio2_flag,po4_flag
index,,,,,
2009,2019-07-24,QNUT5593,BDL,AV,AV
2099,2019-09-04,QNUT5705,BDL,AV,AV
396,2016-05-19,QNUT2164,AV,AV,BDL
411,2016-05-30,QNUT2178,AV,BDL,AV
730,2017-03-30,QNUT2910,SVD,SVD,SVD
...,...,...,...,...,...
3783,2022-03-29,QNUT8248,SVD,AV,SVC
3334,2021-06-22,QNUT7546,SVD,SVD,SVD
3233,2021-04-26,QNUT7424,AV,AV,SVC


# Save Result to Hakai Portal Compatible Excel Format

In [28]:
df_output = df[original_columns]

# Rename Columns to Match Portal Output
# Drop unchanged rows

###ATTENTION### The output has not yet been tested with the Hakai API.
df_output.to_excel('Hakai_Nutrient_Revision_{0}.xlsx'.format(pd.Timestamp.now().isoformat().replace(':','')))


# Report Figures

## Contour plot

In [29]:
# Plot contourf interpolate linearly over the x axis and maximum over two NaN values
def get_contour(var):
    df_pivot = pd.pivot_table(df,values=var,index='line_out_depth',columns='date',aggfunc='mean').interpolate(axis='index',limit=4).sort_index(axis=0).sort_index(axis=1).interpolate(axis='columns',limit=3)
    fig = go.Figure(data =
        go.Contour(z=df_pivot.values,x=df_pivot.columns,y=df_pivot.index.values,
                colorbar=dict(title=var, titleside='right'),
                colorscale='RdYlGn',
                ncontours=10,
                contours_coloring='heatmap'
                #,connectgaps=True
                ))
    fig.update_yaxes(title='Depth (m)',autorange="reversed",linecolor='black',mirror=True,
                    ticks='outside',showline=True)
    fig.update_xaxes(linecolor='black',mirror=True,ticks='outside',showline=True)
    fig.update_layout(width=1500)
    fig.show()

interact(get_contour,var=nutrient_variables)

interactive(children=(Dropdown(description='var', options=('no2_no3_um', 'sio2', 'po4'), value='no2_no3_um'), …

<function __main__.get_contour(var)>

## Scatter with colorbar

In [30]:
## Scatter with colorbar
fig = px.scatter(df.dropna(subset=['no2_no3_um'],axis=0),x='time',y='depth',
                 color='no2_no3_um',color_continuous_scale='RdYlGn',
                 hover_name='hakai_id')
fig.update_yaxes(title='Depth (m)',autorange="reversed",linecolor='black',mirror=True,
                 ticks='outside')
fig.update_xaxes(linecolor='black',mirror=True,ticks='outside',showline=True)
fig.show()

In [31]:
# Add scatter per depth
px.scatter(df.sort_values(['line_out_depth','time']),
           x='time',
           y='no2_no3_um',
           color='depth',
           color_continuous_scale=px.colors.sequential.YlOrRd,
           hover_data=['hakai_id'])